# Interactive maps: Folium & Leaflet

Whenever you go into a website that has some kind of interactive map, it is quite probable that you are wittnessing a map that has been made with a JavaScipt library called [Leaflet](http://leafletjs.com/) (the other popular one that you might have wittnessed is called [OpenLayers](https://openlayers.org/)). 

There is also a Python module called [Folium](https://github.com/python-visualization/folium) that makes it possible visualize data that's been manipulated in Python on an interactive Leaflet map.

 - Let's see how we can use it together with Geopandas

In [3]:
import folium 
import geopandas as gpd
from fiona.crs import from_epsg

# Create a Map instance
map_osm = folium.Map(location=[60.25, 24.8], tiles='Stamen Toner',
                   zoom_start=10, control_scale=True, prefer_canvas=True)

# Create a Clustered map
marker_cluster = folium.MarkerCluster().add_to(map_osm)

# Visualize population grid
fp = r"C:\HY-Data\HENTENKA\KOODIT\Opetus\Automating-GIS-processes\AutoGIS-Sphinx\data\Vaestotietoruudukko_2015\Vaestotietoruudukko_2015.shp"
addr_fp = r"C:\HY-Data\HENTENKA\KOODIT\Opetus\Automating-GIS-processes\AutoGIS-Sphinx\source\data\addresses.shp"
# Read Data 
data = gpd.read_file(fp)
ad = gpd.read_file(addr_fp)

# Re-project to WGS84
data['geometry'] = data['geometry'].to_crs(epsg=4326)
ad['geometry'] = ad['geometry'].to_crs(epsg=4326)

# Update the CRS of the GeoDataFrame
data.crs = from_epsg(4326)
ad.crs = from_epsg(4326)

# Make a selection (only data above 0 and below 1000)
data = data.ix[(data['ASUKKAITA'] > 0) & (data['ASUKKAITA'] <= 1000)]

# Create a Geo-id
data['geoid'] = data.index.astype(str)
ad['geoid'] = ad.index.astype(str)

# Select data
data = data[['geoid', 'ASUKKAITA', 'geometry']]


# Save the file as geojson
jsontxt = data.to_json()

# Create Choropleth map
map_osm.choropleth(geo_str=jsontxt, data=data, columns=['geoid', 'ASUKKAITA'], key_on="feature.id", 
                   fill_color='YlOrRd', fill_opacity=0.9, line_opacity=0.2, line_color='white', line_weight=0,
                   threshold_scale=[100, 250, 500, 1000, 2000], 
                   legend_name='Population in Helsinki', highlight=False, smooth_factor=2.25)


# Create Address points on top of the map
for idx, row in ad.iterrows():
    # Get lat and lon of points
    lon = row['geometry'].x
    lat = row['geometry'].y
    
    # Get address information
    address = row['address']

    #folium.Marker(location=[lat, lon], popup=address).add_to(marker_cluster)
    #folium.Marker(location=[lat, lon], popup=address).add_to(map_osm)
    
    #folium.RegularPolygonMarker(location=[lat, lon], popup=address, fill_color='#2b8cbe', number_of_sides=6, radius=8).add_to(map_osm)
    folium.RegularPolygonMarker(location=[lat, lon], popup=address, fill_color='#2b8cbe', number_of_sides=6, radius=8).add_to(marker_cluster)
    
    #folium.CircleMarker(location=[lat, lon], popup=address, radius=30, color='#3186cc', fill_color='#3186cc').add_to(map_osm)
    
# Save the output
map_osm.save(r'C:\HY-Data\HENTENKA\KOODIT\Opetus\Automating-GIS-processes\Lesson-5-Visualization-Maps\Codes\pop15.html')

# Show it
map_osm